
このノートブックを実行するには、次の追加ライブラリが必要です。 Colab での実行は実験的なものであることに注意してください。問題がある場合は、Github の問題を報告してください。


In [ ]:
!pip install d2l==1.0.0-beta0



# 単語の類似性と類推

:label: `sec_synonyms`

 :numref: `sec_word2vec_pretraining`では、小さなデータセットで word2vec モデルをトレーニングし、それを適用して、入力単語に対して意味的に類似した単語を検索しました。実際には、大規模なコーパスで事前学習された単語ベクトルは、下流の自然言語処理タスクに適用できます。これについては、 :numref: `chap_nlp_app`で後述します。大規模なコーパスからの事前学習済み単語ベクトルのセマンティクスを簡単に示すために、単語の類似性タスクと類推タスクにそれらを適用してみましょう。


In [1]:
import os
import torch
from torch import nn
from d2l import torch as d2l


## 事前学習済みの単語ベクトルの読み込み

以下に、次元 50、100、および 300 の事前トレーニング済み GloVe エンベディングをリストします。これらは、 [GloVe Web サイト](https://nlp.stanford.edu/projects/glove/)からダウンロードできます。事前トレーニングされた fastText 埋め込みは複数の言語で利用できます。ここでは、 [fastText Web サイト](https://fasttext.cc/)からダウンロードできる 1 つの英語版 (300 次元の「wiki.en」) について考えます。


In [2]:
#@save
d2l.DATA_HUB['glove.6b.50d'] = (d2l.DATA_URL + 'glove.6B.50d.zip',
                                '0b8703943ccdb6eb788e6f091b8946e82231bc4d')

#@save
d2l.DATA_HUB['glove.6b.100d'] = (d2l.DATA_URL + 'glove.6B.100d.zip',
                                 'cd43bfb07e44e6f27cbcc7bc9ae3d80284fdaf5a')

#@save
d2l.DATA_HUB['glove.42b.300d'] = (d2l.DATA_URL + 'glove.42B.300d.zip',
                                  'b5116e234e9eb9076672cfeabf5469f3eec904fa')

#@save
d2l.DATA_HUB['wiki.en'] = (d2l.DATA_URL + 'wiki.en.zip',
                           'c1816da3821ae9f43899be655002f6c723e91b88')


これらの事前トレーニング済みの GloVe および fastText 埋め込みをロードするには、次の`TokenEmbedding`クラスを定義します。


In [3]:
#@save
class TokenEmbedding:
    """Token Embedding."""
    def __init__(self, embedding_name):
        self.idx_to_token, self.idx_to_vec = self._load_embedding(
            embedding_name)
        self.unknown_idx = 0
        self.token_to_idx = {token: idx for idx, token in
                             enumerate(self.idx_to_token)}

    def _load_embedding(self, embedding_name):
        idx_to_token, idx_to_vec = ['<unk>'], []
        data_dir = d2l.download_extract(embedding_name)
        # GloVe website: https://nlp.stanford.edu/projects/glove/
        # fastText website: https://fasttext.cc/
        with open(os.path.join(data_dir, 'vec.txt'), 'r') as f:
            for line in f:
                elems = line.rstrip().split(' ')
                token, elems = elems[0], [float(elem) for elem in elems[1:]]
                # Skip header information, such as the top row in fastText
                if len(elems) > 1:
                    idx_to_token.append(token)
                    idx_to_vec.append(elems)
        idx_to_vec = [[0] * len(idx_to_vec[0])] + idx_to_vec
        return idx_to_token, torch.tensor(idx_to_vec)

    def __getitem__(self, tokens):
        indices = [self.token_to_idx.get(token, self.unknown_idx)
                   for token in tokens]
        vecs = self.idx_to_vec[torch.tensor(indices)]
        return vecs

    def __len__(self):
        return len(self.idx_to_token)


以下では、50 次元の GloVe 埋め込み (Wikipedia サブセットで事前トレーニング済み) をロードします。 `TokenEmbedding`インスタンスを作成するとき、指定された埋め込みファイルがまだダウンロードされていない場合はダウンロードする必要があります。


In [4]:
glove_6b50d = TokenEmbedding('glove.6b.50d')


語彙サイズを出力します。語彙には 400,000 の単語 (トークン) と特別な未知のトークンが含まれています。


In [5]:
len(glove_6b50d)

400001


語彙内の単語のインデックスを取得することも、その逆も可能です。


In [6]:
glove_6b50d.token_to_idx['beautiful'], glove_6b50d.idx_to_token[3367]

(3367, 'beautiful')


## 事前学習済みの単語ベクトルの適用

ロードされた GloVe ベクトルを使用して、次の単語の類似性と類似性のタスクに適用することで、そのセマンティクスを実証します。

### 単語の類似性

:numref: `subsec_apply-word-embed`と同様に、単語ベクトル間のコサイン類似性に基づいて入力単語に対して意味的に類似した単語を見つけるために、次の`knn` ($k$-最近傍) 関数を実装します。


In [7]:
def knn(W, x, k):
    # Add 1e-9 for numerical stability
    cos = torch.mv(W, x.reshape(-1,)) / (
        torch.sqrt(torch.sum(W * W, axis=1) + 1e-9) *
        torch.sqrt((x * x).sum()))
    _, topk = torch.topk(cos, k=k)
    return topk, [cos[int(i)] for i in topk]


次に、 `TokenEmbedding`インスタンス`embed`からの事前学習済みの単語ベクトルを使用して、類似した単語を検索します。


In [8]:
def get_similar_tokens(query_token, k, embed):
    topk, cos = knn(embed.idx_to_vec, embed[[query_token]], k + 1)
    for i, c in zip(topk[1:], cos[1:]):  # Exclude the input word
        print(f'cosine sim={float(c):.3f}: {embed.idx_to_token[int(i)]}')


`glove_6b50d`の事前トレーニングされた単語ベクトルの語彙には、400,000 の単語と特別な未知のトークンが含まれています。入力単語と未知のトークンを除いて、この語彙の中から「チップ」という単語に意味的に最も似ている単語を 3 つ見つけてみましょう。


In [9]:
get_similar_tokens('chip', 3, glove_6b50d)

cosine sim=0.856: chips
cosine sim=0.749: intel
cosine sim=0.749: electronics



以下は「baby」と「 beautiful」に似た単語を出力します。


In [10]:
get_similar_tokens('baby', 3, glove_6b50d)

cosine sim=0.839: babies
cosine sim=0.800: boy
cosine sim=0.792: girl


In [11]:
get_similar_tokens('beautiful', 3, glove_6b50d)

cosine sim=0.921: lovely
cosine sim=0.893: gorgeous
cosine sim=0.830: wonderful



### 言葉のたとえ

類似した単語を見つけるだけでなく、単語ベクトルを単語の類推タスクに適用することもできます。たとえば、「男性」:「女性」::「息子」:「娘」は単語の類似形式です。「男性」は「女性」に対して、「息子」は「娘」に対してです。具体的には、単語類似補完タスクは次のように定義できます。単語類似 $a : b :: c : d$ について、最初の 3 つの単語 $a$、$b$、$c$ が与えられた場合、$d$ を見つけます。単語 $w$ のベクトルを $\text{vec}(w)$ で表します。類推を完了するには、$\text{vec}(c)+\text{vec}(b)-\text{vec}(a)$ の結果に最も類似するベクトルを持つ単語を見つけます。


In [12]:
def get_analogy(token_a, token_b, token_c, embed):
    vecs = embed[[token_a, token_b, token_c]]
    x = vecs[1] - vecs[0] + vecs[2]
    topk, cos = knn(embed.idx_to_vec, x, 1)
    return embed.idx_to_token[int(topk[0])]  # Remove unknown words


ロードされた単語ベクトルを使用して、「男性と女性」の類似性を検証してみましょう。


In [13]:
get_analogy('man', 'woman', 'son', glove_6b50d)

'daughter'


以下は「首都と国」の類似例です: 「北京」:「中国」::「東京」:「日本」。これは、事前トレーニングされた単語ベクトルのセマンティクスを示しています。


In [14]:
get_analogy('beijing', 'china', 'tokyo', glove_6b50d)

'japan'


「悪い」:「最悪」::「大きな」:「最大の」などの「形容詞-最上級形容詞」のアナロジーでは、事前トレーニングされた単語ベクトルが構文情報をキャプチャしている可能性があることがわかります。


In [15]:
get_analogy('bad', 'worst', 'big', glove_6b50d)

'biggest'


事前学習済みの単語ベクトルで捕捉された過去時制の概念を示すために、「現在時制 - 過去時制」のアナロジー、つまり「do」:「did」::「go」:「went」を使用して構文をテストできます。


In [16]:
get_analogy('do', 'did', 'go', glove_6b50d)

'went'


## まとめ
- 実際には、大規模なコーパスで事前学習された単語ベクトルを下流の自然言語処理タスクに適用できます。
- 事前トレーニングされた単語ベクトルは、単語の類似性タスクと類推タスクに適用できます。

## 演習
1. `TokenEmbedding(&#39;wiki.en&#39;)`を使用して fastText の結果をテストします。
1. 語彙が非常に多い場合、どうすれば似たような単語を見つけたり、単語の類推をより早く完了したりできるでしょうか?



[ディスカッション](https://discuss.d2l.ai/t/1336)
